In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load data from a single combined CSV file
combined_FD_file = "pca_transformed_data_x.csv"
combined_FD_data = pd.read_csv(combined_FD_file)

# Assuming split_index is defined
split_index = int(0.8 * len(combined_FD_data))  # Adjust as needed

# Split the combined data into X (input) and Y (output) data
X_data = combined_FD_data.iloc[:, :2].values
Y_data = pd.read_csv("MODEL_DATA/NEWDATA/newData_expanded_realHardParam.csv").values

# Standardize the input data
scaler_x = StandardScaler()
X_data = scaler_x.fit_transform(X_data)

# Standardize the target data
scaler_y = StandardScaler()
Y_data = scaler_y.fit_transform(Y_data)

# Define a custom neural network model with L2 regularization
class ComplexNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, lambda_reg):
        super(ComplexNeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        self.layers.append(nn.LeakyReLU(negative_slope=0.01))

        # Hidden layers
        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
            self.layers.append(nn.LeakyReLU(negative_slope=0.01))

        # Output layer
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))

        # Define the regularization strength
        self.lambda_reg = lambda_reg

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def l2_regularization(self):
        l2_reg = 0.0
        for param in self.parameters():
            l2_reg += torch.norm(param) ** 2
        return l2_reg

# Define the L2 regularization strength
lambda_reg = 0.0001
hidden_sizes = [64, 64, 64]
input_size = 2
output_size = 7

# Custom loss function with regularization
class CustomWeightedMSELoss(nn.Module):
    def __init__(self, lambda_reg):
        super(CustomWeightedMSELoss, self).__init__()
        self.lambda_reg = lambda_reg

    def forward(self, outputs, targets):
        errors = outputs - targets
        squared_errors = errors ** 2

        # Calculate the mean squared error
        mse_loss = torch.mean(squared_errors, dim=0)

        # Calculate L2 regularization
        reg_loss = 0.0
        for param in self.parameters():
            reg_loss += torch.norm(param) ** 2

        return torch.sum(mse_loss) + self.lambda_reg * reg_loss

# Create an instance of the neural network
model = ComplexNeuralNetwork(input_size, hidden_sizes, output_size, lambda_reg)
criterion = CustomWeightedMSELoss(lambda_reg=lambda_reg)
optimizer = optim.Adam(model.parameters(), lr=0.00001)  # Initial learning rate

# Convert numpy arrays to torch tensors
X_train = torch.tensor(X_data[:split_index], dtype=torch.float32)
Y_train = torch.tensor(Y_data[:split_index], dtype=torch.float32)
X_val = torch.tensor(X_data[split_index:], dtype=torch.float32)
Y_val = torch.tensor(Y_data[split_index:], dtype=torch.float32)

# Learning rate scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Define early stopping parameters
patience = 20  # Number of consecutive epochs without improvement to tolerate
best_val_loss = float('inf')
no_improvement_count = 0

# Lists to store the training and validation losses for plotting
train_losses = []
val_losses = []

# Training loop with early stopping and learning rate scheduling
num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)

    # Calculate the custom weighted MSE loss with regularization
    loss = criterion(outputs, Y_train)

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation loss
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, Y_val)

    # Update learning rate based on validation loss
    scheduler.step(val_loss)

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')
    
    # Append losses to the lists for later plotting
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_count = 0
        # Save the model when validation loss improves
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print(f'Early stopping at epoch {epoch + 1} as validation loss did not improve for {patience} consecutive epochs.')
        break

# Normalize the input data for prediction
X_test_data = pd.read_csv("MODEL_DATA/NEWDATA/newData_expanded_realHardParam.csv").iloc[:, :2].values
X_test = torch.tensor(scaler_x.transform(X_test_data), dtype=torch.float32)

# Predict parameters
with torch.no_grad():
    predictions = model(X_test)

X_test_denormalized = scaler_x.inverse_transform(X_test.numpy())

# Denormalize the predicted parameters
predicted_params = scaler_y.inverse_transform(predictions.numpy())

param_names = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']

# Print the denormalized predicted parameters
for i, param_name in enumerate(param_names):
    print(f'{param_name}: Predicted = {predicted_params[0][i]:.4f}')



Epoch [100/1000], Loss: 7.2186, Validation Loss: 6.4273
Epoch [200/1000], Loss: 7.1788, Validation Loss: 6.4115
Epoch [300/1000], Loss: 7.1378, Validation Loss: 6.3927
Epoch [400/1000], Loss: 7.0948, Validation Loss: 6.3690
Epoch [500/1000], Loss: 7.0495, Validation Loss: 6.3411
Epoch [600/1000], Loss: 7.0006, Validation Loss: 6.3089
Epoch [700/1000], Loss: 6.9470, Validation Loss: 6.2717
Epoch [800/1000], Loss: 6.8927, Validation Loss: 6.2328
Epoch [900/1000], Loss: 6.8378, Validation Loss: 6.1908
Epoch [1000/1000], Loss: 6.7826, Validation Loss: 6.1468
c1: Predicted = -4.8166
c2: Predicted = 6160.8193
c3: Predicted = -0.0892
c4: Predicted = -0.3050
c5: Predicted = 7339.6196
c6: Predicted = 2050.2737
c7: Predicted = 1074.2635


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load data from a single combined CSV file
combined_FD_file = "MODEL_DATA/NEWDATA/newData_combined_FD.csv"
combined_FD_data = pd.read_csv(combined_FD_file)

# Assuming split_index is defined
split_index = int(0.8 * len(combined_FD_data))  # Adjust as needed

# Split the combined data into X (input) and Y (output) data
X_data = combined_FD_data.iloc[:, :2].values
Y_data = pd.read_csv("MODEL_DATA/NEWDATA/newData_expanded_realHardParam.csv").values

# Define the L2 regularization strength
lambda_reg = 0.0001
hidden_sizes = [64, 64, 64]
output_size = 7

# Standardize the target data
scaler_y = StandardScaler()
Y_data = scaler_y.fit_transform(Y_data)

# Apply PCA to the input data
pca = PCA(n_components=2)
X_data_pca = pca.fit_transform(X_data)

# Standardize the PCA-transformed input data
scaler_x_pca = StandardScaler()
X_data_pca = scaler_x_pca.fit_transform(X_data_pca)

# Create an instance of the neural network
input_size_pca = 2  # Number of principal components
model = ComplexNeuralNetwork(input_size_pca, hidden_sizes, output_size, lambda_reg)
criterion = CustomWeightedMSELoss(lambda_reg=lambda_reg)
optimizer = optim.Adam(model.parameters(), lr=0.000001)  # Initial learning rate

# Convert numpy arrays to torch tensors
X_train = torch.tensor(X_data_pca[:split_index], dtype=torch.float32)
Y_train = torch.tensor(Y_data[:split_index], dtype=torch.float32)
X_val = torch.tensor(X_data_pca[split_index:], dtype=torch.float32)
Y_val = torch.tensor(Y_data[split_index:], dtype=torch.float32)

# Learning rate scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Define early stopping parameters
patience = 20  # Number of consecutive epochs without improvement to tolerate
best_val_loss = float('inf')
no_improvement_count = 0

# Lists to store the training and validation losses for plotting
train_losses = []
val_losses = []

# Training loop with early stopping and learning rate scheduling
num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)

    # Calculate the custom weighted MSE loss with regularization
    loss = criterion(outputs, Y_train)

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation loss
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, Y_val)

    # Update learning rate based on validation loss
    scheduler.step(val_loss)

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

    # Append losses to the lists for later plotting
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_count = 0

    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print(f'Early stopping at epoch {epoch + 1} as validation loss did not improve for {patience} consecutive epochs.')
        break

# Normalize the input data for prediction
X_test_data = pd.read_csv("MODEL_DATA/NEWDATA/newData_expanded_realHardParam.csv").iloc[:, :2].values
X_test = torch.tensor(scaler_x.transform(X_test_data), dtype=torch.float32)

# Predict parameters
with torch.no_grad():
    predictions = model(X_test)

# Denormalize the predicted parameters
predicted_params = scaler_y.inverse_transform(predictions.numpy())

param_names = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']

# Print the denormalized predicted parameters
for i, param_name in enumerate(param_names):
    print(f'{param_name}: Predicted = {predicted_params[0][i]:.4f}')


Epoch 00007: reducing learning rate of group 0 to 1.0000e-07.
Epoch 00013: reducing learning rate of group 0 to 1.0000e-08.
Epoch [100/1000], Loss: 7.2036, Validation Loss: 6.4357
Epoch [200/1000], Loss: 7.2036, Validation Loss: 6.4357
Epoch [300/1000], Loss: 7.2035, Validation Loss: 6.4357
Epoch [400/1000], Loss: 7.2035, Validation Loss: 6.4356
Epoch [500/1000], Loss: 7.2035, Validation Loss: 6.4356
Epoch [600/1000], Loss: 7.2035, Validation Loss: 6.4356
Epoch [700/1000], Loss: 7.2034, Validation Loss: 6.4356
Epoch [800/1000], Loss: 7.2034, Validation Loss: 6.4356
Epoch [900/1000], Loss: 7.2034, Validation Loss: 6.4356
Epoch [1000/1000], Loss: 7.2034, Validation Loss: 6.4356
c1: Predicted = 0.6228
c2: Predicted = 1312.2163
c3: Predicted = 0.1019
c4: Predicted = 1.5112
c5: Predicted = -1780.7350
c6: Predicted = 1486.8063
c7: Predicted = 1319.3363
